### Practice 0808

In [1]:
# using resample to group rows by chunks of time
import pandas as pd
import numpy as np

# create date range
time_index = pd.date_range('06/06/2017', periods=100000, freq='30S')

# create dataframe
dataframe = pd.DataFrame(index=time_index)

In [2]:
# create column of random values
dataframe['sale_amount'] = np.random.randint(1, 10, 100000)

# group rows by week, calculate sum per week
dataframe.resample('W').sum()

,sale_amount
2017-06-11,86814
2017-06-18,100633
2017-06-25,100771
2017-07-02,101531
2017-07-09,100745
2017-07-16,10526


In [3]:
dataframe.head(3)

,sale_amount
2017-06-06 00:00:00,6
2017-06-06 00:00:30,8
2017-06-06 00:01:00,6


In [6]:
# resample the data to two week time periods. Need an aggregation function
# Here we use the mean which will yield the average transaction size over the period
dataframe.resample('2W').mean()

,sale_amount
2017-06-11,5.023958
2017-06-25,4.995139
2017-07-09,5.016766
2017-07-23,5.060577


In [7]:
# here we resample by month and aggregate by count of transactions in period
dataframe.resample('M').count()

,sale_amount
2017-06-30,72000
2017-07-31,28000


#### Looping Over a Column

In [9]:
# you can treat a pandas column like any other sequence in Python
# here we go back to the Titanic data
url = 'https://raw.githubusercontent.com/chrisalbon/sim_data/master/titanic.csv'

# Load data
df = pd.read_csv(url)

In [10]:
# now loop over the name column
for name in df['Name'][0:2]:
    print(name.upper())

ALLEN, MISS ELISABETH WALTON
ALLISON, MISS HELEN LORAINE


In [11]:
# we could also use a list comprehension
[name.upper() for name in df['Name'][0:2]]

['ALLEN, MISS ELISABETH WALTON', 'ALLISON, MISS HELEN LORAINE']

#### Applying a Function Over all Elements in a Column

In [12]:
# you can use the apply method to apply built-in or custom functions
# over all elements in a column

# first make a custom function
def uppercase(x):
    return x.upper()

# apply function, show two rows
df['Name'].apply(uppercase)[0:2]

0    ALLEN, MISS ELISABETH WALTON
1     ALLISON, MISS HELEN LORAINE
Name: Name, dtype: object

In [14]:
# you can use apply as the aggregation function for a groupby
# here we use a basic lambda function to count, but could calculate
# something like a custom statistic
df.groupby('Sex').apply(lambda x: x.count())

,Name,PClass,Age,Sex,Survived,SexCode
Sex,,,,,,
female,462,462,288,462,462,462
male,851,851,468,851,851,851


#### Concatenating DataFrames

In [16]:
# to concatenate two DataFrames you can use the pd.concat() function
# the argument 'axis = 0' tells pandas to concat along the row axis

# Create a DataFrame
data_a = {'id': ['1', '2', '3'], 
          'first': ['Alex', 'Amy', 'Allen'], 
         'last': ['Anderson', 'Ackerman', 'Ali']}
df_a = pd.DataFrame(data_a, columns = ['id', 'first', 'last'])

# Create a second DataFrame
data_b = {'id': ['4', '5', '6'], 
         'first': ['Billy', 'Brian', 'Bran'], 
         'last': ['Bonder', 'Black', 'Balwner']}
df_b = pd.DataFrame(data_b, columns=['id', 'first', 'last'])

In [18]:
pd.concat([df_a, df_b], axis=0)

,id,first,last
0,1,Alex,Anderson
1,2,Amy,Ackerman
2,3,Allen,Ali
0,4,Billy,Bonder
1,5,Brian,Black
2,6,Bran,Balwner


In [19]:
# can use 'axis=1' to concat along the column axis
pd.concat([df_a, df_b], axis=1)

,id,first,last,id,first,last
0,1,Alex,Anderson,4,Billy,Bonder
1,2,Amy,Ackerman,5,Brian,Black
2,3,Allen,Ali,6,Bran,Balwner


In [21]:
# can also use the .append() method to add a new row to a DataFrame
row = pd.Series([10, 'Chris', 'Chillon'], index=['id', 'first', 'last'])

# append row to df_a
df_a.append(row, ignore_index=True)

/var/folders/pd/dk0dsnvs72ng9hv5mkv2d0980000gn/T/ipykernel_12424/2876082931.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_a.append(row, ignore_index=True)


,id,first,last
0,1,Alex,Anderson
1,2,Amy,Ackerman
2,3,Allen,Ali
3,10,Chris,Chillon


#### Merging DataFrames

In [22]:
# if you want to merge two DataFrames you can use the pd.merge(function)
# with various types of joins similar to SQL
# merge defaults to an inner join, otherwise we add the argument 'how'
# Create DataFrame
employee_data = {'employee_id': ['1', '2', '3', '4'],
                 'name': ['Amy Jones', 'Allen Keys', 'Alice Bees',
                 'Tim Horton']}
df_employees = pd.DataFrame(employee_data, columns = ['employee_id',
                                                              'name'])

# Create DataFrame
sales_data = {'employee_id': ['3', '4', '5', '6'],
              'total_sales': [23456, 2512, 2345, 1455]}
df_sales = pd.DataFrame(sales_data, columns = ['employee_id',
                                                      'total_sales'])

pd.merge(df_employees, df_sales, on='employee_id')

# Excerpt From
# Machine Learning with Python Cookbook
# Chris Albon
# https://itunes.apple.com/WebObjects/MZStore.woa/wa/viewBook?id=0
# This material may be protected by copyright.

,employee_id,name,total_sales
0,3,Alice Bees,23456
1,4,Tim Horton,2512


In [23]:
pd.merge(df_employees, df_sales, how='outer', on='employee_id')

,employee_id,name,total_sales
0,1,Amy Jones,NaN
1,2,Allen Keys,NaN
2,3,Alice Bees,23456.0
3,4,Tim Horton,2512.0
4,5,NaN,2345.0
5,6,NaN,1455.0


In [24]:
# can also do left and right joins
# can join on different columns for each df with 'left_on=' and 'right_on='
pd.merge(df_employees, df_sales, how='left', left_on='employee_id', 
        right_on='employee_id')

,employee_id,name,total_sales
0,1,Amy Jones,NaN
1,2,Allen Keys,NaN
2,3,Alice Bees,23456.0
3,4,Tim Horton,2512.0


In [25]:
# can also merge on the indexes of each DataFrame by replacing the
# left_on and right_on parameters with left_index=True and right_index=True
pd.merge(df_employees, df_sales, how='left', left_index=True, 
        right_index=True)

,employee_id_x,name,employee_id_y,total_sales
0,1,Amy Jones,3,23456
1,2,Allen Keys,4,2512
2,3,Alice Bees,5,2345
3,4,Tim Horton,6,1455


##### join type descriptions:

inner: return only the rows that match in both DataFrames(e.g. return any row with an employee_id value appearing in both df_employees and df_sales)

outer: return all rows in both DataFrames. If a row exists in one DataFrame and not in the other, this will fill NaN values for the missing values(e.g. return all rows in both df_employees and df_sales)

left: return all rows from the left DataFrame but only rows from the right DataFrame that matched with the left. Fill NaN values for the missing values(e.g. return all rows from df_employees but only rows from df_sales that have a value for employee_id that appears in df_employees)

right: return all rows from the right DataFrame but only rows from the left DataFrame that matched with the right. Fill NaN values for the missing values(e.g. return all rows from df_sales but only rows from df_employees that have a value for employee_id that appears in df_sales)

